In [197]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [198]:
!pip install pyECLAT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [199]:
!pip install pyfpgrowth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [200]:
# Import headers
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [201]:
#Data preprocessing
dataset = pd.read_csv('/content/Market_Basket_Optimisation.csv', header = 0)
Data=dataset
rows = len(Data.axes[0])
cols = len(Data.axes[1])
Data.columns = range(Data.shape[1])
rows

7500

In [202]:
# Intializing the list
transacts = []
# populating a list of transactions
for i in range(0, rows): 
  transacts.append([str(Data.values[i,j]) for j in range(0, cols)])

In [203]:
#Apriori algorithm
from apyori import apriori
ap = apriori(transactions = transacts, min_support = 0.2, use_colnames=True) # generate rules
# list of results coming from the apriori model
rslt = list(ap)

rslt

[RelationRecord(items=frozenset({'mineral water'}), support=0.23826666666666665, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'mineral water'}), confidence=0.23826666666666665, lift=1.0)]),
 RelationRecord(items=frozenset({'nan'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan'}), confidence=1.0, lift=1.0)]),
 RelationRecord(items=frozenset({'mineral water', 'nan'}), support=0.23826666666666665, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'mineral water', 'nan'}), confidence=0.23826666666666665, lift=1.0), OrderedStatistic(items_base=frozenset({'mineral water'}), items_add=frozenset({'nan'}), confidence=1.0, lift=1.0), OrderedStatistic(items_base=frozenset({'nan'}), items_add=frozenset({'mineral water'}), confidence=0.23826666666666665, lift=1.0)])]

In [221]:
%timeit -n 100 -r 10 ap

100 loops, best of 10: 47.9 ns per loop


In [205]:
#ECLAT algorithm
from pyECLAT import ECLAT

# loading transactions DataFrame to ECLAT class
eclat = ECLAT(data=Data)

# DataFrame of binary values
eclat.df_bin


,NaN,tomato sauce,corn,eggplant,yogurt cake,clothes accessories,toothpaste,burger sauce,meatballs,chili,...,zucchini,bramble,whole wheat rice,pet food,eggs,grated cheese,chocolate bread,whole weat flour,salad,strong cheese
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7497,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [206]:
items_total = eclat.df_bin.astype(int).sum(axis=0)

items_total

NaN                   0
tomato sauce        106
corn                 36
eggplant             99
yogurt cake         205
                   ... 
grated cheese       393
chocolate bread      32
whole weat flour     69
salad                36
strong cheese        58
Length: 121, dtype: int64

In [207]:
# count items in each row
items_per_transaction = eclat.df_bin.astype(int).sum(axis=1)

items_per_transaction

0       3
1       1
2       2
3       5
4       1
       ..
7495    3
7496    6
7497    1
7498    2
7499    4
Length: 7500, dtype: int64

In [208]:
df = pd.DataFrame({'items': items_total.index, 'transactions': items_total.values}) 

# cloning pandas DataFrame for visualization purpose  
df_table = df.sort_values("transactions", ascending=False)

#  Top 5 most popular products/items
df_table.head(5)

,items,transactions
74,mineral water,1787
115,eggs,1348
72,spaghetti,1306
18,french fries,1282
45,chocolate,1229


In [209]:
eclat_indices, eclat_supports = eclat.fit(min_support=0.2)

Combination 1 by 1


1it [00:00, 47.11it/s]


Combination 2 by 2


0it [00:00, ?it/s]


Combination 3 by 3


0it [00:00, ?it/s]


In [210]:
len(eclat_supports)

1

In [211]:
print(eclat_supports)

{'mineral water': 0.23826666666666665}


In [269]:
%timeit -n 100 -r 10 eclat_supports

100 loops, best of 10: 32.3 ns per loop


In [213]:
import pyfpgrowth

In [214]:
min_support= 0.2*rows
patterns = pyfpgrowth.find_frequent_patterns(transacts, min_support)
patterns

{('mineral water',): 1787,
 ('mineral water', 'nan'): 25383,
 ('nan',): 120657,
 ('nan', 'nan'): 941741}

In [275]:
%timeit -n 100 -r 10 patterns

100 loops, best of 10: 31.8 ns per loop
